# Setup

These should already be installed in your notebook environment.
You can uncomment and run if needed

In [1]:
#!pip install -qU elasticsearch
!pip install -qU eland[pytorch]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.3/523.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 1.5.3 which is incompatible.
xarray 2024.6.0 requires pandas>=2.0, but you have pandas 1.5.3 which is incompatible.


Import the required python libraries

In [ ]:
import os
from elasticsearch import Elasticsearch, helpers, exceptions
from urllib.request import urlopen
from getpass import getpass
import json
import time

Create an Elasticsearch Python client

In [ ]:
es = Elasticsearch(
    hosts = ["http://kubernetes-vm:9200"],
    basic_auth=("elastic", "changeme")
)

# Upload Hugging Face model with Eland

Use Eland's `eland_import_hub_model` command to upload the model to Elasticsearch.

For this example we've chosen the [`cross-encoder/ms-marco-MiniLM-L-6-v2`](https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2) text similarity model.

In [ ]:
MODEL_ID = "cross-encoder/ms-marco-MiniLM-L-6-v2"

!eland_import_hub_model \
  --url "http://kubernetes-vm:9200" \
  -u "elastic" \
  -p "changeme" \
  --hub-model-id $MODEL_ID \
  --task-type text_similarity

# Create Inference Endpoint
We need to create an endpoint queries can use for reranking

In [ ]:
model_config = {
  "service": "elasticsearch",
  "service_settings": {
    "num_allocations": 1,
    "num_threads": 1,
    "model_id": "cross-encoder__ms-marco-minilm-l-6-v2"
  },
      "task_settings": {
        "return_documents": True
    }
}

inference_id = "semantic-reranking"

create_endpoint = es.inference.put(
    inference_id=inference_id,
    task_type="rerank",
    body=model_config
)

create_endpoint.body

Verify it was created

You should see JSON output with information about the semantic endpoint

In [ ]:
check_endpoint = es.inference.get(
    inference_id=inference_id,
)

check_endpoint.body

{'endpoints': [{'model_id': 'semantic-reranking',
   'inference_id': 'semantic-reranking',
   'task_type': 'rerank',
   'service': 'elasticsearch',
   'service_settings': {'num_allocations': 8,
    'num_threads': 1,
    'model_id': 'cross-encoder__ms-marco-minilm-l-6-v2'},
   'task_settings': {'return_documents': True}}]}

Verify the model was successfully deployed

The cell below should return 'started'




In [ ]:
ES_MODEL_ID = "cross-encoder__ms-marco-minilm-l-6-v2"

model_info = es.ml.get_trained_models_stats(model_id=ES_MODEL_ID)

model_info.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']['routing_state']

# Test query with reranking


In [ ]:
USER_QUESTION = "Where can I get good pizza?"

response = es.search(
    index="rstaurant_reviews",
    body={
      "retriever": {
        "text_similarity_reranker": {
          "retriever": {
            "standard": {
              "query": {
                "match": {
                  "review": USER_QUESTION
                }
              }
            }
          },
          "field": "review",
          "inference_id": "semantic-reranking",
          "inference_text": USER_QUESTION,
          "rank_window_size": 20,
          "min_score": 0.4
        }
      },
      "_source": "false",
      "fields": [
        "review",
        "restaurant"
      ]
    }
)